In [ ]:
import pandas as pd
import geopandas as gpd

#the airbnb list
airbnb = pd.read_csv("result.csv", encoding='utf-8')
# print(airbnb.shape)
#the urban heritage list
p = pd.read_csv("gcj02towgs84ed listing.csv", encoding='utf-8')
# print(p.shape)
result = airbnb[airbnb['keyword']!='0']
result = result.reset_index(drop=True)
result['keyword'] = [tuple(i.split(",")) for i in result['keyword']]

#calculte the number of listings located within a 2km radius of the specified urban heritage and their id
def p_listings(n):
    #Input the number of national urban heritages
    #Output a subset of 10 listings located within a 2km radius of the heritages
    x = [p.loc[n,'new_x']]
    y = [p.loc[n,'new_y']]
    s = gpd.GeoSeries.from_xy(x, y, crs="EPSG:4490")
    s = s.to_crs(4548) #for changing the unit to meter
    s_b = s.buffer(2000) 
    s = gpd.GeoDataFrame(s)
    s = s.rename(columns={0:'geometry'}).set_geometry('geometry')
    s_b = gpd.GeoDataFrame(s_b)
    s_b = s_b.rename(columns={0:'geometry'}).set_geometry('geometry')
    airbnb_g2 = airbnb_g.to_crs(4548)
    out_a = gpd.sjoin(airbnb_g2,s_b,how = "inner")
    out_a = out_a.reset_index(drop=True)
    
    listings_id = []
    for n in range(len(out_a)):
        listings_id.append(int(out_a.loc[n,'id']))
    l_amount = len(listings_id)
    
    return listings_id, l_amount

##### calculate
p['2km_id'] = 0
p['2km_amount'] = 0
for n in range(len(p)):
    l_i, l_a = p_listings(n)
    p.loc[n,'2km_id'] = str(l_i)
    p.loc[n,'2km_amount'] = l_a
    
#calculte the number of listings mentioned the specified urban heritage in 2km
p['m_id'] = 0
for i in range(len(p)):
    ls = []
    for j in range(len(airbnb)):
        if p.loc[i,'listing'] in airbnb.loc[j,'keyword']:
            ls.append(int(airbnb.loc[j,'id']))

    p.loc[i,'m_id'] = str(ls)
p['m_amount'] = [len(i.split(",")) for i in p['m_id']]
p['2km_m_id'] = 0
p['2km_m_amount'] = 0
for i in range(len(p)):
    if p.loc[i,'m_id'] == '[]':
        p.loc[i,'m_amount'] = 0
        
    km_i = p.loc[i,'2km_id'][1:-1].split(',')
    m_i = p.loc[i,'m_id'][1:-1].split(',')
    km_m_i = list(set(km_i) & set(m_i))
    km_m_a = len(km_m_i)
    if p.loc[i,'2km_id']== '[]':
        km_m_a = 0
        km_m_i = []
    p.loc[i,'2km_m_id'] = str(km_m_i)
    p.loc[i,'2km_m_amount'] = km_m_a
    if p.loc[i,'2km_amount']==0 & p.loc[i,'2km_m_amount']>0:
        p.loc[i,'2km_m_percent'] = 200.01
    elif p.loc[i,'2km_m_amount']==0:
        p.loc[i,'2km_m_percent'] = 0
    else:
        p.loc[i,'2km_m_percent'] = round(p.loc[i,'2km_m_amount']/p.loc[i,'2km_amount'],4)*100

#define function to attain the subdataset of airbnb listings which mentioned the specified urban heritage
result['id'] = [str(int(i)) for i in result['id']]
def p_l(n):
    l = p.loc[n,'m_id'][1:-1].split(', ')
    listp = []
    for ll in l:
        a = result[result['id']==ll]
        a = a.reset_index(drop=True)
        listp.append(a)
    
    p_l_sub = pd.concat(listp).reset_index(drop=True)
    return p_l_sub

# calculate the indicators related to district
for j in range(len(p)):
    if p.loc[j,'m_amount'] > 0:
        sub = p_l(j)
        district_s =  p.loc[j,'district']
        sub_s = sub[sub['district']== district_s]#lisitngs in the same district
        p.loc[j,'sd_m_amount'] = len(sub_s)
        p.loc[j,'sd_m_percent_in_d'] = round(p.loc[j,'sd_m_amount']/len(airbnb[airbnb['district']==district_s])*100,2)
        p.loc[j,'sd_m_percent_in_m'] = round(p.loc[j,'sd_m_amount']/p.loc[j,'m_amount']*100,2)
        
    else:
        p.loc[j,'sd_m_amount'] = 0
        p.loc[j,'sd_m_percent_in_d'] = 0.00
        p.loc[j,'sd_m_percent_in_m'] = 0.00
        
        
p.to_csv("listing_final.csv", index=False, index_label=False) 